In [2]:
import CAT
import sys
import json
import logging
import numpy as np
import pandas as pd

In [3]:
def setuplogger():
    root = logging.getLogger()
    root.setLevel(logging.INFO)
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter("[%(levelname)s %(asctime)s] %(message)s")
    handler.setFormatter(formatter)
    root.addHandler(handler)

In [4]:
setuplogger()

In [5]:
# choose dataset here
dataset = 'assistment'
# modify config here
config = {
    'learning_rate': 0.002,
    'batch_size': 2048,
    'num_epochs': 10,
    'num_dim': 10, # for IRT or MIRT
    'device': 'cpu',
    # for NeuralCD
    'prednet_len1': 128,
    'prednet_len2': 64,
}

In [6]:
# read datasets
train_triplets = pd.read_csv(f'../data/{dataset}/train_triples.csv', encoding='utf-8').to_records(index=False)
concept_map = json.load(open(f'../data/{dataset}/concept_map.json', 'r'))
concept_map = {int(k):v for k,v in concept_map.items()}
metadata = json.load(open(f'../data/{dataset}/metadata.json', 'r'))

In [7]:
train_data = CAT.dataset.TrainDataset(train_triplets, concept_map,
                                      metadata['num_train_students'], 
                                      metadata['num_questions'], 
                                      metadata['num_concepts'])

In [8]:
# define model here
model = CAT.model.IRTModel(**config)
# train model
model.init_model(train_data)
model.train(train_data, log_step=10)

[INFO 2021-03-09 14:46:27,480] train on cpu
[INFO 2021-03-09 14:46:27,579] Epoch [1] Batch [0]: loss=inf
[INFO 2021-03-09 14:46:27,920] Epoch [1] Batch [10]: loss=0.76105
[INFO 2021-03-09 14:46:28,267] Epoch [1] Batch [20]: loss=0.72569
[INFO 2021-03-09 14:46:28,482] Epoch [2] Batch [0]: loss=inf
[INFO 2021-03-09 14:46:28,826] Epoch [2] Batch [10]: loss=0.75531
[INFO 2021-03-09 14:46:29,177] Epoch [2] Batch [20]: loss=0.72002
[INFO 2021-03-09 14:46:29,391] Epoch [3] Batch [0]: loss=inf
[INFO 2021-03-09 14:46:29,732] Epoch [3] Batch [10]: loss=0.74957
[INFO 2021-03-09 14:46:30,080] Epoch [3] Batch [20]: loss=0.71441
[INFO 2021-03-09 14:46:30,242] Epoch [4] Batch [0]: loss=inf
[INFO 2021-03-09 14:46:30,638] Epoch [4] Batch [10]: loss=0.74343
[INFO 2021-03-09 14:46:30,969] Epoch [4] Batch [20]: loss=0.70825
[INFO 2021-03-09 14:46:31,129] Epoch [5] Batch [0]: loss=inf
[INFO 2021-03-09 14:46:31,505] Epoch [5] Batch [10]: loss=0.73560
[INFO 2021-03-09 14:46:31,855] Epoch [5] Batch [20]: loss

In [9]:
# save model
model.adaptest_save('../ckpt/mirt.pt')